In [ ]:
import pandas as pd

In [ ]:
# download up-to-date csv file from https://instances.vantage.sh/
# filtering can be done before download, if preferable
# either set path, or put csv in your working directory (sample names given)
new_csv = "Amazon EC2 Instance Comparison (1).csv"
df = pd.read_csv(new_csv)
existing_csv = "Amazon EC2 Instance Comparison.csv"
old_df = pd.read_csv(existing_csv)

In [ ]:
# make sure it looks okay
df.head()

In [ ]:
df = df.set_index('API Name')

In [ ]:
# the generation field is useful to identify current EC2s
# but unnecessary once they have been removed
df.drop("Generation", axis=1, inplace=True)

In [ ]:
idx_lst = old_df["API Name"].tolist()

In [ ]:
# take the subset of EC2s which are present in old/exisiting table
# this ensures the other fields for each EC2 are up-to-date
new_df = df[df.index.isin(idx_lst)]

In [ ]:
idx_new = new_df.index.tolist()

In [ ]:
# see which entries will be removed
set(idx_lst) - set(idx_new)

In [ ]:
# Test filter to observe entries to add
# df.filter(like='r6i',axis=0)

In [ ]:
# not efficient, but a non-issue for small sheets
# use two search terms "prefix" and "suffix" of API Name
add_list = ['r6i.', 'r6id.', 'r6a', 'c6a', 'c6i', 'm6i', 'm6a']
keep = ['.large','.xlarge','2x','4x','8x','16x','32x']
add = first_pass = df.filter(like='xyz', axis=0)
for ec2 in add_list:
    first_pass = df.filter(like=ec2,axis=0)
    for size in keep:
        add = pd.concat([add,first_pass.filter(like=size, axis=0)])
print(len(set(add)))
print(add)

In [ ]:
# index should be reset to include Name field in output
new_df = new_df.reset_index().set_index('Name')
add = add.reset_index().set_index('Name')

In [ ]:
final_df = pd.concat([new_df,add])

In [ ]:
final_df.sort_values(by='Instance Memory',axis=0,inplace=True)

In [ ]:
# check the output
final_df.head(20)

In [ ]:
# avoid overwriting the existing sheet
final_df.to_csv("New Amazon EC2 Instance Comparison.csv")